In [1]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    pipeline,
    logging,
)
import pandas as pd
import os

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

hf_token = os.environ.get('hf_token')

# Need auth token for these
hf_auth = hf_token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

## Table data

In [9]:
import json
from tablular_eval_util import combine_csv_files, split_dataframe, preprocess_data, run_llm_inference

directory = '/home/hb/dataset_bgp/bgp_tab_dataset_test'
combined_df = combine_csv_files(directory)

# Split the DataFrame into smaller chunks
split_size = 10
data_list = split_dataframe(combined_df, split_size)

# Preprocess the data into the required format
formatted_data = [preprocess_data(chunk) for chunk in data_list]

formatted_data_file = 'llm_table_bgp_data_test_10.json'
with open(formatted_data_file, 'w') as f:
    json.dump(formatted_data, f, indent=4)

with open(formatted_data_file, 'r') as f:
    formatted_data = json.load(f)

output_results_file = 'llm_13B_bgp_data_with_outputs_10.json'
run_llm_inference(formatted_data, model, tokenizer, max_length=3050, output_results_file=output_results_file)

Processed 1/20
Processed 2/20
Processed 3/20
Processed 4/20
Processed 5/20
Processed 6/20
Processed 7/20
Processed 8/20
Processed 9/20
Processed 10/20


/home/hb/myenv/lib/python3.8/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Processed 11/20
Processed 12/20
Processed 13/20
Processed 14/20
Processed 15/20
Processed 16/20
Processed 17/20
Processed 18/20
Processed 19/20
Processed 20/20
[{'instruction': 'The goal for this task is to determine if the data indicates an anomaly. The context, section, and table columns provide important information for identifying the correct anomaly type.', 'input_seg': '[TLE] The context is about BGP data analysis for detecting anomalies. The section is related to a specific time period of BGP monitoring. [TAB] col: | timestamp | asn | num_routes | num_new_routes | num_withdrawals | num_origin_changes | num_route_changes | max_path_length | avg_path_length | max_edit_distance | avg_edit_distance | num_announcements | num_unique_prefixes_announced | anomaly_status | row 1: | 2022-03-28 07:00:00 | 8342 | 0 | 0 | 0 | 0 | 0 | 0 | 0.0 | 0 | 0.0 | 0 | 0 | no anomalies detected | [SEP] row 2: | 2022-03-28 07:05:00 | 8342 | 7 | 7 | 0 | 0 | 0 | 0 | 0.0 | 0 | 0.0 | 19 | 7 | no anomalies de

In [10]:
import pandas as pd
import re

llama13b_outputs = pd.read_json("/home/hb/LLM-research/LLM_testing/llm_13B_bgp_data_with_outputs_10.json")
llama13b_outputs = llama13b_outputs["output"]
# llama13b_outputs.to_csv("llm_13B_bgp_data_with_outputs_10.csv")

timestamps = []

timestamp_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}')

for output in llama13b_outputs:
    matches = timestamp_pattern.findall(output)
    timestamps.extend(matches)

llama13b_outputs_timestamp = list(set(timestamps))

print(llama13b_outputs_timestamp)

['2022-03-28 10:35:00', '2022-03-28 11:00:00', '2017-12-12 07:10:00', '2022-03-28 14:25:00', '2019-05-08 15:05:00', '2017-12-12 07:05:00', '2022-03-28 14:05:00', '2022-03-28 13:25:00', '2022-03-28 16:05:00', '2022-03-28 10:15:00', '2022-03-28 14:15:00', '2022-03-28 13:35:00', '2022-03-28 10:30:00', '2022-03-28 11:40:00', '2022-03-28 11:35:00', '2022-03-28 13:30:00', '2017-12-12 04:45:00', '2022-03-28 14:20:00', '2022-03-28 15:55:00', '2019-05-08 15:10:00', '2022-03-28 10:25:00', '2017-12-12 04:40:00', '2022-03-28 10:40:00', '2022-03-28 10:45:00', '2022-03-28 10:55:00', '2022-03-28 14:00:00']


In [11]:
import pandas as pd
from tablular_eval_util import combine_csv_files

directory = '/home/hb/dataset_bgp/bgp_tab_dataset_test'
combined_df = combine_csv_files(directory)
combined_df = combined_df[['anomaly_status']]

# Filter the rows with "anomaly detected" in the anomaly_status
filtered_df = combined_df[combined_df['anomaly_status'].str.contains('anomaly detected', na=False)]
filtered_df.to_csv('/home/hb/dataset_bgp/bgp_tab_dataset_test/test_true_label.csv', index=False)

# Extract the date from the anomaly_status string
true_label = filtered_df['anomaly_status'].str.extract(r'anomaly detected at (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

# Convert the dates to a list
true_label = true_label[0].tolist()

print(true_label)

['2022-03-28 10:25:00', '2022-03-28 10:30:00', '2022-03-28 10:35:00', '2022-03-28 10:40:00', '2022-03-28 10:55:00', '2022-03-28 11:00:00', '2022-03-28 11:35:00', '2022-03-28 11:40:00', '2022-03-28 11:45:00', '2022-03-28 13:25:00', '2022-03-28 13:30:00', '2022-03-28 13:35:00', '2022-03-28 14:00:00', '2022-03-28 14:05:00', '2022-03-28 14:15:00', '2022-03-28 14:20:00', '2022-03-28 14:25:00', '2022-03-28 15:55:00', '2022-03-28 16:00:00', '2022-03-28 16:05:00', '2019-05-08 15:05:00', '2019-05-08 15:10:00', '2019-05-08 15:15:00', '2019-05-08 15:30:00', '2017-12-12 04:40:00', '2017-12-12 04:45:00', '2017-12-12 04:50:00', '2017-12-12 07:05:00', '2017-12-12 07:10:00']


In [12]:
import pandas as pd
from tablular_eval_util import evaluate_llm_results

evaluation_result = evaluate_llm_results(true_anomalies=true_label, llm_results=llama13b_outputs_timestamp)
print("Evaluation Results:")
print(f"Precision: {evaluation_result['precision']:.2f}")
print(f"Recall: {evaluation_result['recall']:.2f}")
print(f"F1 Score: {evaluation_result['f1_score']:.2f}")
print(f"True Positives: {evaluation_result['true_positives']}")
print(f"False Positives: {evaluation_result['false_positives']}")
print(f"False Negatives: {evaluation_result['false_negatives']}")

Evaluation Results:
Precision: 0.92
Recall: 0.83
F1 Score: 0.87
True Positives: 24
False Positives: 2
False Negatives: 5


# Pcap

## DDoS

In [3]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create a Python script with the Scapy library to detect DDoS attacks from a PCAP file. The script should read the file, analyze traffic patterns by monitoring packets per second against a set threshold within defined time windows, and reset counters for each time window. It should also log any potential attacks showing the source IP and packet rate. Include a final check for the last time window at the end of the file."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=712)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Create a Python script with the Scapy library to detect DDoS attacks from a PCAP file. The script should read the file, analyze traffic patterns by monitoring packets per second against a set threshold within defined time windows, and reset counters for each time window. It should also log any potential attacks showing the source IP and packet rate. Include a final check for the last time window at the end of the file. [/INST]  Sure! Here is a Python script using the Scapy library to detect DDoS attacks from a PCAP file:
```
import scapy
import time
import datetime

# Set parameters
pcap_file = "attack_capture.pcap"
threshold = 1000 # packets per second
time_window = 10 # seconds

# Open PCAP file
pcap = scapy.PCAP(pcap_file)

# Initialize counters
source_ip_counter = {}
packet_counter = {}

# Loop through PCAP file
for p in pcap:
    # Check if packet is from a new source IP
    if p.src not in source_ip_counter:
        # Initialize source IP counter
        source_ip_count

In [3]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Given a n3.pcap file containing network traffic data, create a Python script using scapy library to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1012)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Given a n3.pcap file containing network traffic data, create a Python script using scapy library to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it. [/INST]  Sure! Here is a Python script using the Scapy library to process a PCAP file and extract the source IP, destination IP, source port, destination port, and raw payload data for each packet. The data is then stored in a pandas DataFrame and displayed.
```
import scapy
import pandas as pd

# Open the PCAP file
pcap = scapy.open("n3.pcap")

# Create a list to store the packet data
packet_data = []

# Iterate over each packet in the PCAP file
for packet in pcap:
    # Extract the source IP, destination IP, source port, and destination port
    src_ip = packet.src_ip
    dst_ip = packet.dst_ip
    src_port = packet.sport
    dst_port = packet.dport
    
    # Extra

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create a Python script using PyBGPStream to monitor real-time (project='ris-live') BGP updates, focusing on announcements from AS6830. The script utilizes a dictionary to track the timestamp and count of announcements for each prefix associated with AS6830. Implement a function that checks daily if the data needs to be reset by comparing the current date with the date of the last reset, and if they differ, it clears the dictionary, ensuring data freshness each new day. The script should also alert if any prefix's count exceeds 10,000 to flag significant anomalies in real time. For extracting AS paths from BGP updates, use the code as_path = elem.fields.get('as-path', '').split() to accurately parse the path and identify announcements directly associated with AS6830."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=718)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

# 5G General Knowledge

In [ ]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/5G/5G_analysis_multi_choice_v2.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer. {question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
            if line.startswith("Answer: ") or line.startswith("Correct answer: "):
                match = re.search(r'\b(\w)[\)-]', line)
                if match is not None:
                    parsed_answer = match.group(1)
                    print(f"Parsed_answer: {parsed_answer}")
                    break
                else:
                    parsed_answer = "Couldn't parse the answer"
                    print(f"Parsed_answer: {parsed_answer}")
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

In [ ]:
import json
input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_SPEC_QA.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

# 5G analysis knowledge

## Multiple Choice
`5G configuration analysis v1`:
Accuracy: 90%
Correct answers: 21 + 7 = 28
Incorrect answers: 2

`5G configuration analysis v2`:
Accuracy: 70.00%
Correct answers: 21
Incorrect answers: 12